In [10]:
import findspark
findspark.init()

# Creamos la session de Spark
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()
print('Apache Spark Version :' + spark.sparkContext.version)

Apache Spark Version :3.3.1


In [11]:
df_json = spark.read.text("file_fallido.csv")

In [3]:
# # Structura
# json_schema_df = spark.read.json(df.rdd.map(lambda row: row.value))
# json_schema = json_schema_df.schema
# json_schema

StructType([StructField('resource', ArrayType(StructType([StructField('fecha', StringType(), True), StructField('id_ponsa', StringType(), True)]), True), True)])

In [12]:
json_schema = StructType([StructField('resource', ArrayType(
        StructType([StructField('fecha', StringType(), True), 
                    StructField('id_ponsa', StringType(), True)]), True), True)])

In [13]:
#Apply the schema to payload to read the data
df_details = df_json.withColumn("parsed_data", from_json(df_json["value"], json_schema)).drop("value")

In [14]:
df_details.printSchema()

root
 |-- parsed_data: struct (nullable = true)
 |    |-- resource: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- fecha: string (nullable = true)
 |    |    |    |-- id_ponsa: string (nullable = true)



In [15]:
#Bajamas un nivel para no hacer 'parsed_data.columna'
df = df_details.select(col("parsed_data.*"))
df.printSchema()

root
 |-- resource: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- fecha: string (nullable = true)
 |    |    |-- id_ponsa: string (nullable = true)



In [16]:
print("EXPLOIDE delays")
df = df.withColumn("rs", explode_outer("resource"))

EXPLOIDE delays


In [17]:
df.select('*').show()

+--------+----+
|resource|  rs|
+--------+----+
|    null|null|
+--------+----+

